# Project Assignment B
Link to git repository: https://github.com/ongiboy/Cognitive-Social-Science

Group members:
* Christian Ong Hansen (s204109)
* Kavus Latifi Yaghin (s214601)
* Daniel Damkjær Ries (s214641)

Group member's contribution:
* Every task was made in collaboration by all members.

## 1. Motivation

* What is your dataset
* Why did you choose this/these particular dataset(s)?
* What was your goal for the end user’s experience?

## 2. Basic stats
* Write about your choices in data cleaning and preprocessing
* Write a short section that discusses the dataset stats

### Data retrieval and cleaning/preprocessing

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
import requests
import itertools
import networkx as nx
from collections import defaultdict
from functools import partial
from tqdm import tqdm
from requests_futures.sessions import FuturesSession
import matplotlib.pyplot as plt
import ast
import numpy as np
from statistics import mode
import json

headers = {"accept": "application/json"}
api_key = "f5813332cb558d374cbcb057ea2fc48b"

In [ ]:
def movie_titles_and_IDs_from_actor_ID(actor_id, session):
    url = f"https://api.themoviedb.org/3/person/{actor_id}/movie_credits?api_key={api_key}"
    response = session.get(url)
    data = response.json()
    # Return the whole movie dictionary, not just the title
    return [movie['title'] for movie in data['cast']], [movie['id'] for movie in data['cast']]

In [ ]:
def fetch_page(page, session):
    url = f"https://api.themoviedb.org/3/person/popular?api_key={api_key}&page={page}"
    response = session.get(url)
    data = response.json()
    people = []
    for person in data['results']:
        person_url = f"https://api.themoviedb.org/3/person/{person['id']}?api_key={api_key}"
        person_response = session.get(person_url)
        person_data = person_response.json()
        people.append((person['name'], person['id'], person_data['gender'], person_data['birthday'], person_data['place_of_birth']))
    return people

with requests.Session() as session:
    session.headers.update(headers)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        pages = list(range(1, 251))
        fetch_page_with_session = partial(fetch_page, session=session)
        people = list(executor.map(fetch_page_with_session, pages))

all_people_names, all_people_ids, all_people_genders, all_people_birthdays, all_people_birthplaces = zip(*itertools.chain(*people))

In [ ]:
# Create a DataFrame with 'actors' column
df_actors = pd.DataFrame(all_people_names, columns=['actors'])

# Add 'ids', 'genders', and 'birthplaces' columns to the DataFrame
df_actors['actor_ids'] = all_people_ids
df_actors['genders'] = all_people_genders
df_actors['ages'] = all_people_birthdays # This is not the age, but the birthday
df_actors['birthplaces'] = all_people_birthplaces

# Change birthplaces so that it only contains the country (text after the last comma)
df_actors['birthplaces'] = df_actors['birthplaces'].str.split(',').str[-1]

# Change birthday to age
df_actors['ages'] = pd.to_datetime(df_actors['ages'], errors='coerce')
df_actors['ages'] = (pd.to_datetime('today') - df_actors['ages']).dt.days // 365

# Drop rows with missing values
df_actors.dropna(inplace=True)

# Drop duplicates
df_actors.drop_duplicates(subset='actors', inplace=True)
#reset index
df_actors.reset_index(drop=True, inplace=True)


In [ ]:
# Fetch movies for each actor and add them to 'movies' and 'movie_IDs' columns
with requests.Session() as session:
    session.headers.update(headers)
    movie_titles_and_ids_from_actor_ID_with_session = partial(movie_titles_and_IDs_from_actor_ID, session=session)
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        movies_and_ids = list(executor.map(movie_titles_and_ids_from_actor_ID_with_session, df_actors['actor_ids']))

# Add 'movies' column to df_actors
df_actors['movies'] = [x[0] for x in movies_and_ids]

# Flatten movies_and_ids into two separate lists
movies = [item for sublist in [x[0] for x in movies_and_ids] for item in sublist]
ids = [item for sublist in [x[1] for x in movies_and_ids] for item in sublist]

# Convert the lists into a list of dictionaries
movies_and_ids_dict = [{'movie': movie, 'movie_ID': id} for movie, id in zip(movies, ids)]

# Convert the list of dictionaries into a DataFrame
movies_df = pd.DataFrame(movies_and_ids_dict)
movies_df = movies_df.drop_duplicates()
movies_df.reset_index(drop=True, inplace=True)

In [ ]:
def fetch(session, url):
    future = session.get(url, headers=headers)
    return future

def fetch_all(urls):
    with FuturesSession() as session:
        futures = [fetch(session, url) for url in urls]
        responses = [future.result().json() for future in futures]
    return responses

In [ ]:
# Prepare the URLs
urls = [f"https://api.themoviedb.org/3/movie/{id}?api_key={api_key}" for id in movies_df["movie_ID"]]
print('urls are prepared')

# Fetch all responses
responses = fetch_all(urls)
print('all responses are fetched')

# Process the responses
for i, response in enumerate(responses):
    if i % 100 == 0:
        print(f"Processing response {i+1}/{len(responses)}")
    if isinstance(response, Exception):
        print(f"Error: {response}")
        continue  # Skip this response
    # Process the response here

# Initialize empty lists to store the data
ratings = []
popularities = []
genres = []
release_dates = []
abstracts = []

# Process the responses one by one
for data in tqdm(responses):
    ratings.append(data.get('vote_average'))
    popularities.append(data.get('popularity'))
    genres.append([genre['name'] for genre in data.get('genres', [])])
    release_dates.append(data.get('release_date'))
    abstracts.append(data.get('overview'))
    
# Assign the lists to the DataFrame columns
movies_df['rating'] = ratings
movies_df['popularity'] = popularities
movies_df['genres'] = genres
movies_df['release_date'] = release_dates
movies_df['abstract'] = abstracts

In [ ]:
# Drop duplicates
movies_df.drop_duplicates(subset='movie', inplace=True)

# Drop rows with missing values
movies_df.dropna(inplace=True)

# Remove rows with empty lists in 'genres' column
movies_df = movies_df[movies_df['genres'].apply(lambda x: len(x) > 0)]

# Remove rows with empty release dates
movies_df = movies_df[movies_df['release_date'].apply(lambda x: len(x) > 0)]

# Drop row if abstract is missing
movies_df = movies_df[movies_df['abstract'].apply(lambda x: len(x) > 0)]

# Drop row if rating is missing
movies_df = movies_df[movies_df['rating'].apply(lambda x: x > 0)]

# Drop row if popularity is missing
movies_df = movies_df[movies_df['popularity'].apply(lambda x: x > 0)]

# Reset index
movies_df.reset_index(drop=True, inplace=True)

In [ ]:
# Combine the two dataframes so that each actor is associated with the movies they have acted in
df_actors_filtered = df_actors.copy()
df_movies_filtered = movies_df.copy()
df_movies_filtered.rename(columns={'movie': 'movies'}, inplace=True)

df_actors_filtered = df_actors_filtered.explode('movies').reset_index(drop=True)

df_actors_movies = df_actors_filtered.merge(df_movies_filtered, on='movies', how='left')

# Remove all rows where the release date is before 2010 or after 2024 (to avoid movies that have not been released yet)
df_actors_movies = df_actors_movies[(df_actors_movies['release_date'] >= '2010-01-01') & (df_actors_movies['release_date'] <= '2024-04-16')]

# Collapse the the actors in the actor column so there is only one row per actor and the movies and movie_IDs are stored in lists
df_actors_filtered = df_actors_movies.groupby('actors').agg({'actor_ids': 'first',
                                                             'genders': 'first',
                                                             'birthplaces': 'first',
                                                             'ages': 'first',
                                                             'movies': list, 
                                                             'movie_ID': list}).reset_index()

df_movies_filtered = df_actors_movies.drop_duplicates(subset=['movies']).groupby('movies').agg({'movie_ID': 'first',
                                                                                               'rating': 'first',
                                                                                               'popularity': 'first',
                                                                                               'genres': 'first',
                                                                                               'release_date': 'first',
                                                                                               'abstract': 'first'}).reset_index()

Cleaning the birthplace column, to avoid having the same country spelled differently etc...

In [ ]:
import pycountry
import re

# Clean birthplace column in actor dataframe
len_bef_clean = len(df_actors_filtered['birthplaces'].unique())

country_names = [country.name for country in pycountry.countries]

def normalize_country_name(name):
    # strip name
    name = name.strip()
    # replace '.' with ''
    name = name.replace('.', '').replace(']', '')

    if "UK" in name or "İngiltere" in name:
        return "United Kingdom"
    try:
        # Try to get the country object
        country = pycountry.countries.get(name=name)
        if country is not None:
            # If the country object is found, return the official name
            return country.name
        else:
            # If the country object is not found, try to find it by its common name
            country = pycountry.countries.search_fuzzy(name)
            return country[0].name
    except LookupError:
        # Standardizing names
        for country_name in country_names:
            if country_name in name:
                name = country_name

        # Fixing abbreviations and wird instances
        if "Russia" in name:
            return "Russian Federation"
        elif "Türkiye" in name or "Turkey" in name:
            return "Türkiye"
        elif "USA" in name or " US" in name or "United States" in name:
            return "United States"
        elif "Korea" in name:
            return "Korea, Republic of"
        elif "Czech" in name:
            return "Czechia"
        # Hardcoded, could use package to translate, maybe not necessary, few occurences
        elif "TX" in name:
            return "United States"
        elif "Frankrike" in name:
            return "France"
        elif "Afrique du Sud" in name:
            return "South Africa"
        elif "Irlanda" in name:
            return "Ireland"
        elif "中国" in name or "中华民国" in name or "重庆" in name or "南京" in name:
            return "China"
        
        
        # Updating from old names
        if "now" in name:
            match = re.search(r'\[now (.*?)', name)
            if match:
                return match.group(1)

        # If the country is not found, return the original name
        return name

# Normalize the country names in the DataFrame
df_actors_filtered['birthplaces'] = df_actors_filtered['birthplaces'].apply(normalize_country_name)

len_aft_clean = len(df_actors_filtered['birthplaces'].unique())
print("length before cleaning", len_bef_clean)
print("length after cleaning", len_aft_clean)

In [ ]:
# Set all popularity scores above 100 to 100, to mitigate the effect of outliers
df_movies_filtered['popularity'] = df_movies_filtered['popularity'].clip(upper=100)

In [ ]:
# Calculate the histogram
counts, bin_edges = np.histogram(df_movies_filtered['popularity'], bins=100)

# Add a small constant to counts to avoid log(0)
counts = counts + 1e-10

# Plot the histogram
plt.plot(bin_edges[:-1], counts)

# Set the scale of both axes to logarithmic
plt.xscale('log')
plt.yscale('log')

plt.show()

In [ ]:
plt.hist(df_movies_filtered['popularity'], bins=100)
plt.show()

In [ ]:
# Plot the age distribution, NOTHINGS DONE YET
df_actors_filtered['ages'].hist(bins=30, edgecolor='white')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.cm as cm

# Get the counts of each unique value in the 'birthplace' column
birthplace_counts = df_actors_filtered['birthplaces'].value_counts()

# Calculate the percentage of each birthplace out of the total number of actors
birthplace_percentages = (birthplace_counts / df_actors_filtered['birthplaces'].count()) * 100

# Get the top 5 birthplaces
top_5_birthplaces = birthplace_percentages[:10]

# Create a color map
colors = cm.rainbow(np.linspace(0, 1, len(top_5_birthplaces)))

# Create a bar plot of the top 5 birthplaces with different colors for each bar
top_5_birthplaces.plot(kind='bar', color=colors, edgecolor='black')

plt.title('Top 10 Birthplaces of Actors')
plt.xlabel('Birthplace')
plt.ylabel('Percentage of Actors (%)')

plt.show()

print("The number of unique birthplaces:",len(df_actors_filtered['birthplaces'].unique()))
# Get the birthplaces with a count of 1
birthplaces_with_count_1 = birthplace_counts[birthplace_counts == 1]

print("Number of countries only associated with One actor in our dataset",len(birthplaces_with_count_1))

Note: TMBD very biased towards USA with their popular actors data that we have retrieved (over 50% of the total number of actors). Therefore most of the movies are american and the actors from different countries are biased towards the american movie industry.

It is also worth noting that 45 of the 109 unique birthplaces in our dataset is only associated with one actor, which is also a factor causing the unexpected low assortativity for the birthplace attribute. 

To fix this or make it more representative, we could webscrape from other websites to make a less biased dataset.

In [ ]:
# save the dataframes to csv files
df_actors_filtered.to_csv('data/actors.csv', index=False)
df_movies_filtered.to_csv('data/movies.csv', index=False)

### Creating the network

In [ ]:
# load the dataframes from csv files
df_actors_filtered = pd.read_csv('data/actors.csv', converters={'movies': ast.literal_eval, 'movie_ID': ast.literal_eval})
df_movies_filtered = pd.read_csv('data/movies.csv', converters={'genres': ast.literal_eval})

In [ ]:
# Step 1: Prepare the Data
movie_to_actors = defaultdict(list)

for _ , row in df_actors_filtered.iterrows():
    actor = row['actors']
    movies = row['movies']
    for movie in movies:
        movie_to_actors[movie].append(actor)

edges = defaultdict(int)
for actors in movie_to_actors.values():
    for pair in itertools.combinations(sorted(actors), 2):
        edges[pair] += 1

# Step 2: Create the Graph
G = nx.Graph()

# Add nodes with attributes
for _ , row in df_actors_filtered.iterrows():
    G.add_node(row['actors'], gender=row['genders'], age=row['ages'], birthplace=row['birthplaces'])

# Add edges with weights
for edge, weight in edges.items():

# YAHNI
    if weight >= 2:
        G.add_edge(*edge, weight=weight)

In [ ]:
# Find number of nodes and edges
N = G.number_of_nodes()
L = G.number_of_edges()

print("Number of nodes: ", N)
print("Number of edges: ", L)

## 3. Tools, theory and analysis
* Talk about how you’ve worked with text, including regular expressions, unicode, etc.

* Describe which network science tools and data analysis strategies you’ve used, how those network science measures work, and why the tools you’ve chosen are right for the problem you’re solving.

* How did you use the tools to understand your dataset?

##### Degrees and weights

In [ ]:

# Compute node degrees
degrees = dict(G.degree())
weighted_degrees = dict(G.degree(weight='weight'))

# Compute the specificied values
average_degree = np.mean(list(degrees.values()))
median_degree = np.median(list(degrees.values()))
try:
    mode_degree = mode(list(degrees.values())) #Degree value that occurs with highest frequency among the nodes
except:
    mode_degree = "No unique mode"
min_degree = min(degrees.values())
max_degree = max(degrees.values())

# Same calculations but for STRENGTH (WEIGHTED DEGREE)
average_weighted_degree = np.mean(list(weighted_degrees.values()))
median_weighted_degree = np.median(list(weighted_degrees.values()))
try:
    mode_weighted_degree = mode(list(weighted_degrees.values()))
except:
    mode_weighted_degree = "No unique mode"
min_weighted_degree = min(weighted_degrees.values())
max_weighted_degree = max(weighted_degrees.values())

print("Degree Analysis:")
print(f"Average Degree: {average_degree}")
print(f"Median Degree: {median_degree}")
print(f"Mode Degree: {mode_degree}")
print(f"Minimum Degree: {min_degree}")
print(f"Maximum Degree: {max_degree}")

print("\nWeighted Degree Analysis:")
print(f"Average Weighted Degree: {average_weighted_degree}")
print(f"Median Weighted Degree: {median_weighted_degree}")
print(f"Mode Weighted Degree: {mode_weighted_degree}")
print(f"Minimum Weighted Degree: {min_weighted_degree}")
print(f"Maximum Weighted Degree: {max_weighted_degree}")

In [ ]:
# Sort the edges by weight in descending order
sorted_edges = sorted(G.edges.data(), key=lambda x: x[2]['weight'], reverse=True)
most_connected_actors = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
most_connected_actors_weighted = sorted(weighted_degrees.items(), key=lambda x: x[1], reverse=True)

print("Top 5 actors with the highest degree:")
for i, actor in enumerate(most_connected_actors[:5], start=1):
    print(f"Rank: {i}, Actor: {actor[0]}, Degree: {actor[1]}")

print("\nTop 5 actors with the highest weighted degree:")
for i, actor in enumerate(most_connected_actors_weighted[:5], start=1):
    print(f"Rank: {i}, Actor: {actor[0]}, Weighted Degree: {actor[1]}")

print("\nTop 5 most important edges:")
for i, edge in enumerate(sorted_edges[:5], start=1):
    print(f"Rank: {i}, Edge: {edge[0]} - {edge[1]}, Weight: {edge[2]['weight']}")


These are the top ten actors who have participated the most together, as the weight of an edge represents the number of times two actors (nodes) have co-acted in movies together. 

In [ ]:
# Calculate degree distribution
degree_sequence = sorted([d for n, d in G.degree()], reverse=False)  # sort in ascending order
degree_count = nx.degree_histogram(G)

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# degree distribution
axs[0].bar(range(len(degree_count)), degree_count, align='center')
axs[0].axvline(average_degree, color='r', linestyle='-', label='Average Degree')
axs[0].set_xlabel('Degree')
axs[0].set_ylabel('Count')
axs[0].set_title('Degree Distribution')
axs[0].legend()

# loglog degree distribution
axs[1].bar(range(len(degree_count)), degree_count, align='center')
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[1].set_xlabel('Degree')
axs[1].set_ylabel('Count')
axs[1].set_title('Degree Distribution (log scale)')
axs[1].axvline(average_degree, color='r', linestyle='-', label='Average Degree')
axs[1].legend()

plt.show()

In [ ]:
# Compute the probability that makes the expected number of edges equal to the actual number of edges in the graph.
p = 2*L/(N*(N-1))
print("Prob:",p)

# Compute the natural logarithm of N
ln_N = np.log(N)

print(f"ln(N): {ln_N:.2f} < k: {average_degree:.2f}\nAnd p: {p:.5f} > ln(N)/N: {ln_N/N:.5f}")

As the average degree is 10.22 and thus <k> > 1, and it is larger than ln(N) while p > ln(N)/N, the network must fall into the connected regime. It is therefore above the critical threshold. However, as we have thresholded the network by removing edges with an weight less than 2, it is not fully connected.

In [ ]:
# Plot with netwulf
import netwulf as nw

#nw.visualize(G)


In [ ]:
# Calculate the degree assortativity coefficient
degree_assortativity = nx.degree_assortativity_coefficient(G)

# Calculate the attribute assortativity coefficient
country_assortativity = nx.attribute_assortativity_coefficient(G, 'birthplace')

gender_assortativity = nx.attribute_assortativity_coefficient(G, 'gender')

# Calculate the numeric assortativity coefficient
age_assortativity = nx.numeric_assortativity_coefficient(G, 'age')


print(f"degree_assortativity: {degree_assortativity}, country_assortativity: {country_assortativity}, gender_assortativity: {gender_assortativity}")

print(f"age_assortativity: {age_assortativity}")

node to country assortativity: expected a higher value, but the litl lower value can be due to several reasons, desired nationality diversity, moving to different countries (living place) etc...

In [ ]:
# Get the isolates
isolates = list(nx.isolates(G))

# Print the number of isolates
print(len(isolates))

In [ ]:
# x = df_movies_filtered['rating']
# x2 = df_movies_filtered['popularity']
# plt.scatter(x,x2) #ad

In [ ]:
# Calculate the closeness centrality of the network
closeness_centrality = nx.closeness_centrality(G)

# Sort the actors according to the closeness centrality
sorted_closeness_centrality = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# Find the 5 most central actors
most_central_actors = sorted_closeness_centrality[:10]
most_central_actors

In [ ]:
# Calculate the eigenvector centrality of the network
eigenvector_centrality = nx.eigenvector_centrality(G)

# Sort the scientists according to the eigenvector centrality
sorted_eigenvector_centrality = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# Find the 5 most central actors
most_central_actors2 = sorted_eigenvector_centrality[:10]
most_central_actors2

In [ ]:
# Plot the closeness centrality vs node degree to see if there is a correlation
closeness_centrality_values = list(closeness_centrality.values())
eigenvector_centrality_values = list(eigenvector_centrality.values())
degree_values = list(dict(G.degree()).values())

fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# plot the closeness centrality vs node degree to see if there is a correlation
axs[0].scatter(degree_values, closeness_centrality_values, alpha=0.5)
axs[0].set_xlabel('Node Degree')
axs[0].set_ylabel('Closeness Centrality')
axs[0].set_title('Closeness Centrality vs Node Degree')

# plot the eigenvector centrality vs node degree to see if there is a correlation
axs[1].scatter(degree_values, eigenvector_centrality_values, alpha=0.5)
axs[1].set_xlabel('Node Degree')
axs[1].set_ylabel('Eigenvector Centrality')
axs[1].set_title('Eigenvector Centrality vs Node Degree')

plt.show()

### NLP TIME

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import word_tokenize, MWETokenizer
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
# Define a function to tokenize and clean text
def tokenize_and_clean_text3(text, collocations): #takes text and dictionary of collocations
    # Exclude URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Exclude mathematical symbols and numbers
    text = re.sub(r'\b\d+\b', '', text)
    # Exclude punctuation and convert to lowercase
    text = re.sub(r'\W', ' ', text).lower()

    # Tokenize
    tokenizer = MWETokenizer(list(collocations.keys()))
    tokens = word_tokenize(text)
    
    tokens = tokenizer.tokenize(tokens)
    # stem
    tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english')]# TO REMOVE STOPWORDS 
    return tokens

In [ ]:
# Assume abstracts is a list of movie abstracts
abstracts = df_movies_filtered['abstract'].copy().tolist()

# Tokenize the abstracts and remove stopwords
tokens = [word for abstract in abstracts for word in word_tokenize(abstract.lower()) if word not in stopwords.words('english')] # maybe delete, also in function

# Find bigram collocations
bigram_finder = BigramCollocationFinder.from_words(tokens)
bigram_measures = BigramAssocMeasures()

# Get the top 1000 bigrams by the Pointwise Mutual Information measure
top_bigrams = bigram_finder.nbest(bigram_measures.pmi, 1000)

# Convert the list of tuples into a dictionary
collocations = defaultdict(int)
for bigram in top_bigrams:
    collocations[bigram] += 1

In [ ]:
df_movies_filtered['tokens'] = df_movies_filtered['abstract'].apply(lambda x: tokenize_and_clean_text3(x, collocations=collocations))

In [ ]:
import community as community_louvain
# Compute the best partition using the Louvain method
partition = community_louvain.best_partition(G)

# Compute the modularity of this partition
modularity = community_louvain.modularity(partition, G)

print("Modularity found by Louvain algorithm:", modularity)

In [ ]:
# Calculate the number of communities and their sizes
communities = {}
for node, community in partition.items():
    if community not in communities:
        communities[community] = []
    communities[community].append(node)

num_communities = len(communities)
community_sizes = sorted([len(nodes) for nodes in communities.values()], reverse=True)

print("Number of communities:", num_communities)
print("Sizes of communities:", community_sizes)
# Check if the modularity is significantly different than 0
if abs(modularity) > 0.01:  # or any other threshold you consider significant
    print("The modularity is significantly different than 0.")
else:
    print("The modularity is not significantly different than 0.")

In [ ]:
# Create a DataFrame from the partition and degree information
communities_df = pd.DataFrame({
    'actors': list(G.nodes),
    'community': [partition[node] for node in G.nodes],
    'degree': [G.degree(node) for node in G.nodes]
})

In [ ]:
# Merge actors_works_df with communities_df
df = pd.merge(df_actors_filtered, communities_df, on='actors')
df = df.drop(columns=["movies"]).explode('movie_ID')


# Merge with abstracts_df
df = pd.merge(df, df_movies_filtered, left_on='movie_ID', right_on='movie_ID')

# # Get abstract tokens for all communities
all_communities_abstracts = df.groupby('community')['tokens'].agg("sum").reset_index()

all_communities_abstracts.columns = ['Community', 'Abstract Tokens']

In [ ]:
from collections import Counter
# The top 5 communities
top5_communities = communities_df['community'].value_counts().nlargest(5).index

# Filter dataframe for top 5 communities
top5_communities_abstracts = all_communities_abstracts[all_communities_abstracts['Community'].isin(top5_communities)]


top_terms = {}
for community in top5_communities:
    # Get the abstract tokens for the community
    abstract_tokens = top5_communities_abstracts[top5_communities_abstracts['Community'] == community]['Abstract Tokens'].values[0]
    # Get the TF of each term
    term_counts = Counter(abstract_tokens)
    # top 5 most frequent terms
    top_terms[community] = term_counts.most_common(5)

In [ ]:
for item in top_terms.items():
    print(f"Community {item[0]}:")
    print(f"Top 5 terms: {item[1]}\n")

In [ ]:
import math
from tqdm import tqdm
# Get the top 9 communities and filter the abstracts for these communities
top9_communities = communities_df['community'].value_counts().nlargest(9).index
top9_communities_abstracts = all_communities_abstracts[all_communities_abstracts['Community'].isin(top9_communities)]

top_terms = {}
top_tfidf_terms = {}
all_tfidf_terms = {}

# (for less computational cost) get IDF for each term once and store the results
idf_dict = {}
for term in tqdm(set.union(*top9_communities_abstracts['Abstract Tokens'].apply(set))):
    idf_dict[term] = math.log(len(top9_communities_abstracts) / 
                              sum(term in abstract for abstract in top9_communities_abstracts['Abstract Tokens']))


for community in tqdm(top9_communities):
    # Get abstract tokens for the community
    abstract_tokens = top9_communities_abstracts[top9_communities_abstracts['Community'] == community]['Abstract Tokens'].values[0]
    
    # The top 10 TF terms
    term_counts = Counter(abstract_tokens)
    top_terms[community] = term_counts.most_common(10)


    tfidf = {}
    for term, count in term_counts.items():
        # TF for all terms in top 9 communities
        tf = count / len(abstract_tokens)

        # Get IDF from the precalculated dictionary
        idf = idf_dict[term]
        
        # TF-IDF for all terms in top 9 communities
        tfidf[term] = tf * idf

    all_tfidf_terms[community] = sorted(tfidf.items(), key=lambda x: x[1], reverse=True)
    
    # Get the top 10 TF-IDF words
    top_tfidf_terms[community] = sorted(tfidf.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
for item in top_tfidf_terms.items():
    print(f"Community {item[0]}:")
    print(f"Top 10 TF-IDF terms: {item[1]}\n")

In [ ]:
top9_communities_df = communities_df[communities_df['community'].isin(top9_communities)]

# Group by Community and Actor, and sum the Degree to get the total Degree for each Actor in each Community
grouped_df = top9_communities_df.groupby(['community', 'actors'])['degree'].sum().reset_index()

# Make new dataframe storing the top 3 actors by degree for each community
top_actors = grouped_df.groupby('community').apply(lambda x: x.nlargest(5, 'degree')).reset_index(drop=True)
top_actors

In [ ]:
# load files for wordcloud
top_actors = pd.read_csv('data/top_actors.csv')

top9_communities = pd.read_csv('data/top9_communities.csv')

with open('data/all_tfidf_terms.json', 'r') as file:
    all_tfidf_terms = json.load(file)


In [ ]:
from wordcloud import WordCloud
fig, axs = plt.subplots(3, 3, figsize=(24, 24))

for i, community in enumerate(top9_communities["community"]):
    # Get the already calculated top TF-IDF terms for the community
    tfidf_terms = dict(all_tfidf_terms[str(community)])
    
    # Create the word cloud
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = None, 
                min_font_size = 10).generate_from_frequencies(tfidf_terms)
    
    # Subplot indices
    row = i // 3
    col = i % 3

    # Plot the word cloud
    axs[row, col].imshow(wordcloud)
    axs[row, col].axis("off")

    # Get the names of the top three actors in the community
    top_three_actors = top_actors[top_actors['community'] == community]['actors'].values
    # Create the title string with newlines
    title_string = "\n".join(top_three_actors)

    # Set title of subplot to the names of the top three actors, each on a new line
    axs[row, col].set_title(f"Top actors in community {community}:\n{title_string}", fontsize=15)

# Remove any extra subplots
for j in range(i+1, 9):
    fig.delaxes(axs.flatten()[j])

plt.tight_layout(pad = 1) 
plt.subplots_adjust(hspace = 0.25)  # Increase the hspace value
plt.show()

In [ ]:
import json
import pandas as pd

# Save all_tfidf_terms as a JSON file in the data folder
with open('data/all_tfidf_terms.json', 'w') as f:
    json.dump(all_tfidf_terms, f)

# Save top9_communities as a CSV file in the data folder
pd.DataFrame(top9_communities, columns=['community']).to_csv('data/top9_communities.csv', index=False)

# Save top_actors as a CSV file in the data folder
top_actors.to_csv('data/top_actors.csv', index=False)

## 4. Discussion
* What went well?
* What is still missing? What could be improved? Why?